In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
import os
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd

In [3]:
scratch_cache_dir = "/mnt/fast0/rje41/.cache/huggingface"    
model_path = "meta-llama/Llama-3.2-1B-Instruct"
os.makedirs(os.path.join(scratch_cache_dir, "hub"), exist_ok=True)
os.makedirs(os.path.join(scratch_cache_dir, "datasets"), exist_ok=True)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             cache_dir=os.path.join(scratch_cache_dir, "hub")
                                            )

In [5]:
model.config.use_cache = False 
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [6]:
lora_config = LoraConfig(
   r=64,
   lora_alpha=128,
   target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
   lora_dropout=0.05,
   bias="none",
   task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 13,631,488 || all params: 1,249,445,888 || trainable%: 1.0910


In [9]:
from datasets import Dataset

In [22]:
file_path = '../../datasets/add_sub.csv'

df = pd.read_csv(file_path)
dataset = Dataset.from_pandas(df)

def concat_clean_label(example):
    example["clean_label"] = example["clean"] + str(example["label"])
    return example

dataset = dataset.map(concat_clean_label)

def create_labels_no_mask(example):
    tokenized = tokenizer(example["clean_label"], truncation=True, padding="max_length", max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

dataset = dataset.map(create_labels_no_mask, batched=False, remove_columns=['clean', 'label', 'corrupted', 'clean_label'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir='./logs',
    logging_steps=1,
)

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [27]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: rjeskins (rjeskins-university-of-bath) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


KeyboardInterrupt: 